In [1]:
import numpy as np 
import pandas as pd 

# pandas options
pd.set_option('display.max_columns', 50)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('mode.use_inf_as_na', True)
pd.options.mode.chained_assignment = None

# for date manipulation
from datetime import datetime

# for visualization: matplotlib
from matplotlib import pyplot as plt
from IPython.core.pylabtools import figsize
%matplotlib inline
# to display visuals in the notebook

# for visualization: seaborn
import seaborn as sns
sns.set_context(font_scale=2)

# for feature engineering: itertools
from itertools import combinations

# for data preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectFromModel

# for building the model and calculate RMSE
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt

# for hyperparamter optimization
from hyperopt import hp, tpe
from hyperopt.fmin import fmin

import os

In [2]:
# load train  data
building = pd.read_csv("building_metadata.csv")
weather_train = pd.read_csv("weather_train.csv")
train = pd.read_csv("train.csv")

In [3]:
# Reducing the Memory size of the pandas dataframes helps in easier computation 

def reduce_memory_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    print('Mem. usage before the memory reduction: {:5.2f} Mb'.format(start_mem))
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
building = reduce_memory_usage(building)
weather_train = reduce_memory_usage(weather_train)
train = reduce_memory_usage(train)

Mem. usage before the memory reduction:  0.07 Mb
Mem. usage decreased to  0.03 Mb (60.3% reduction)
Mem. usage before the memory reduction:  9.60 Mb
Mem. usage decreased to  3.07 Mb (68.1% reduction)
Mem. usage before the memory reduction: 616.95 Mb
Mem. usage decreased to 289.19 Mb (53.1% reduction)


In [5]:
# Recap from last notebook --> features like square feet have impact on energy consumption 
# some features like wind direction have almost no effect 

In [6]:
# Feature Engineering and Selection 
# We have 15 possible columns for to use for the potential feature set. 
# 15 is a low number compared to the 40 million rows in the test set. 
# few number of features might end up in underfitting  

# Merging 3 dataframes into one and then apply feature generation 
# --> transformation of numerical features into categorical --> wind_direction --> primary use 
# Extract features from existing columns --> age of the building 
# From timestamp --> month, hour, day of the month, day of the month,
# day of the week, is_weekend

# After feature generation -- > person's coefficients of the features to the
# target variable and look at the collinearity between the features to 
# decide on the final set.


In [7]:
# Merging dataframes 

train = train.merge(building, on = "building_id", how = "left")
train = train.merge(weather_train, on = ["site_id", "timestamp"])

# convert timestamp column to date time data type column
train["timestamp"] = pd.to_datetime(train["timestamp"],
                                   format='%Y-%m-%d %H:%M:%S')
train.shape

(20125605, 16)

In [8]:
train.head(10)

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01,0.000,0,Education,7432,2008.000,nan,25.000,6.000,20.000,nan,1019.500,0.000,0.000
1,1,0,2016-01-01,0.000,0,Education,2720,2004.000,nan,25.000,6.000,20.000,nan,1019.500,0.000,0.000
2,2,0,2016-01-01,0.000,0,Education,5376,1991.000,nan,25.000,6.000,20.000,nan,1019.500,0.000,0.000
3,3,0,2016-01-01,0.000,0,Education,23685,2002.000,nan,25.000,6.000,20.000,nan,1019.500,0.000,0.000
4,4,0,2016-01-01,0.000,0,Education,116607,1975.000,nan,25.000,6.000,20.000,nan,1019.500,0.000,0.000
5,5,0,2016-01-01,0.000,0,Education,8000,2000.000,nan,25.000,6.000,20.000,nan,1019.500,0.000,0.000
6,6,0,2016-01-01,0.000,0,Lodging/residential,27926,1981.000,nan,25.000,6.000,20.000,nan,1019.500,0.000,0.000
7,7,0,2016-01-01,0.000,0,Education,121074,1989.000,nan,25.000,6.000,20.000,nan,1019.500,0.000,0.000
8,8,0,2016-01-01,0.000,0,Education,60809,2003.000,nan,25.000,6.000,20.000,nan,1019.500,0.000,0.000
9,9,0,2016-01-01,0.000,0,Office,27000,2010.000,nan,25.000,6.000,20.000,nan,1019.500,0.000,0.000


In [9]:
## Feature Generation 

In [10]:
# wind_direction changing numerical feature into categorical feature   

def convert_direction(series):
    if series <= 90:
        return 0
    # as norteast direction
    elif series <= 180:
        return 1
    # as southeast direction
    elif series <= 270:
        return 2
    # as southwest direction
    elif series <= 360:
        return 3
    # as northwest direction


In [11]:
train['wind_compass_direction'] = train.wind_direction.apply(convert_direction)
train.drop(columns=['wind_direction'], inplace=True)

In [12]:
train.iloc[200:205]

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_speed,wind_compass_direction
200,173,1,2016-01-01,19.343,2,Education,90903,nan,nan,15.602,6.000,-5.602,nan,1015.500,3.600,2.000
201,174,0,2016-01-01,179.900,2,Education,90900,nan,nan,15.602,6.000,-5.602,nan,1015.500,3.600,2.000
202,174,1,2016-01-01,52.858,2,Education,90900,nan,nan,15.602,6.000,-5.602,nan,1015.500,3.600,2.000
203,175,0,2016-01-01,86.590,2,Education,111635,1989.000,nan,15.602,6.000,-5.602,nan,1015.500,3.600,2.000
204,175,1,2016-01-01,116.337,2,Education,111635,1989.000,nan,15.602,6.000,-5.602,nan,1015.500,3.600,2.000


In [13]:
 # Primary use 
# create label encoder object and transform the column
le = LabelEncoder()
le_primary_use = le.fit_transform(train.primary_use)

# add label encoded column to dataframe
train['le_primary_use'] = le_primary_use

In [14]:
train.iloc[2000:2005]

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_speed,wind_compass_direction,le_primary_use
2000,667,0,2016-01-01 01:00:00,2.000,5,Entertainment/public assembly,15715,1966.000,2.000,5.000,0.000,2.000,nan,nan,2.600,2.000,1
2001,668,0,2016-01-01 01:00:00,29.800,5,Entertainment/public assembly,47275,1976.000,2.000,5.000,0.000,2.000,nan,nan,2.600,2.000,1
2002,669,0,2016-01-01 01:00:00,6.800,5,Entertainment/public assembly,4768,1976.000,1.000,5.000,0.000,2.000,nan,nan,2.600,2.000,1
2003,670,0,2016-01-01 01:00:00,14.900,5,Healthcare,18471,1966.000,1.000,5.000,0.000,2.000,nan,nan,2.600,2.000,3
2004,671,0,2016-01-01 01:00:00,2.900,5,Lodging/residential,7589,1976.000,3.000,5.000,0.000,2.000,nan,nan,2.600,2.000,4


In [15]:
# Extract features from existing columns 

# add building age column
current_year = datetime.now().year
train['building_age'] = current_year - train['year_built']
train.drop(columns=['year_built'], inplace=True)
train.shape

(20125605, 17)

In [16]:
train.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_speed,wind_compass_direction,le_primary_use,building_age
0,0,0,2016-01-01,0.000,0,Education,7432,nan,25.000,6.000,20.000,nan,1019.500,0.000,0.000,0,11.000
1,1,0,2016-01-01,0.000,0,Education,2720,nan,25.000,6.000,20.000,nan,1019.500,0.000,0.000,0,15.000
2,2,0,2016-01-01,0.000,0,Education,5376,nan,25.000,6.000,20.000,nan,1019.500,0.000,0.000,0,28.000
3,3,0,2016-01-01,0.000,0,Education,23685,nan,25.000,6.000,20.000,nan,1019.500,0.000,0.000,0,17.000
4,4,0,2016-01-01,0.000,0,Education,116607,nan,25.000,6.000,20.000,nan,1019.500,0.000,0.000,0,44.000


In [17]:
# Time related data from timestamp 
# Recall that some seasonality factors in the time-series data especially 
# for the chilled and hot water consumption

# Changing the timestamp to month, day_of_week, day_of_the_month 
# and hour columns instead of timestamp will help the model catch 
# the seasonality impact without using the timestamp column

# add month, day of week, day of month and hour
train['month'] = train['timestamp'].dt.month.astype(np.int8)
train['day_of_week'] = train['timestamp'].dt.dayofweek.astype(np.int8)
train['day_of_month']= train['timestamp'].dt.day.astype(np.int8)
train["hour"] = train["timestamp"].dt.hour
# add is weekend column
train['is_weekend'] = train.day_of_week.apply(lambda x: 1 if x>=5 else 0)


In [18]:
train.shape

(20125605, 22)

In [19]:
train.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_speed,wind_compass_direction,le_primary_use,building_age,month,day_of_week,day_of_month,hour,is_weekend
0,0,0,2016-01-01,0.000,0,Education,7432,nan,25.000,6.000,20.000,nan,1019.500,0.000,0.000,0,11.000,1,4,1,0,0
1,1,0,2016-01-01,0.000,0,Education,2720,nan,25.000,6.000,20.000,nan,1019.500,0.000,0.000,0,15.000,1,4,1,0,0
2,2,0,2016-01-01,0.000,0,Education,5376,nan,25.000,6.000,20.000,nan,1019.500,0.000,0.000,0,28.000,1,4,1,0,0
3,3,0,2016-01-01,0.000,0,Education,23685,nan,25.000,6.000,20.000,nan,1019.500,0.000,0.000,0,17.000,1,4,1,0,0
4,4,0,2016-01-01,0.000,0,Education,116607,nan,25.000,6.000,20.000,nan,1019.500,0.000,0.000,0,44.000,1,4,1,0,0


In [20]:
correlations_transformed = pd.DataFrame(train.corr())
correlations_transformed = pd.DataFrame(correlations_transformed["meter_reading"]).reset_index()

In [21]:
# format, and display sorted correlations_transformed
correlations_transformed.columns = ["Feature", "Correlation with meter_reading"]
correlations_transformed = (correlations_transformed[correlations_transformed["Feature"] != "meter_reading"]
                .sort_values(by="Correlation with meter_reading", ascending=True))
display(correlations_transformed)

,Feature,Correlation with meter_reading
14,building_age,-0.113
13,le_primary_use,-0.010
15,month,-0.007
10,sea_level_pressure,-0.004
6,air_temperature,-0.004
8,dew_temperature,-0.003
19,is_weekend,-0.001
16,day_of_week,-0.001
12,wind_compass_direction,-0.000
17,day_of_month,-0.000


In [22]:
# Strongest correlationis -0.1 to 0.1 not as high as expected shifting to log_meter_readings
# add log_meter_reading column to the dataframe
train['log_meter_reading'] = np.log1p(train.meter_reading)


In [23]:
correlations_transformed = pd.DataFrame(train.corr())
correlations_transformed = pd.DataFrame(correlations_transformed["log_meter_reading"]).reset_index()

# format, and display sorted correlations_transformed
correlations_transformed.columns = ["Feature", "Correlation with log_meter_reading"]
correlations_transformed = (correlations_transformed[correlations_transformed["Feature"] != "log_meter_reading"]
                .sort_values(by="Correlation with log_meter_reading", ascending=True))
display(correlations_transformed)

,Feature,Correlation with log_meter_reading
14,building_age,-0.104
13,le_primary_use,-0.063
7,cloud_coverage,-0.033
19,is_weekend,-0.033
11,wind_speed,-0.032
16,day_of_week,-0.026
12,wind_compass_direction,-0.011
10,sea_level_pressure,-0.009
6,air_temperature,-0.005
9,precip_depth_1_hr,0.003


In [24]:
# Feature Selection 
# throwing away precip_depth_1_hr 

# 20 features chosen 
feature_set = ['building_age', 'le_primary_use', 'cloud_coverage',
               'is_weekend','wind_speed', 'day_of_week',
               'wind_compass_direction', 'sea_level_pressure', 'air_temperature',
               'day_of_month', 'dew_temperature', 'hour', 
               'month', 'meter', 'building_id', 
               'site_id', 'floor_count', 'square_feet']


In [25]:
X = train[feature_set]

In [26]:
X.shape

(20125605, 18)

In [27]:
X.head()

,building_age,le_primary_use,cloud_coverage,is_weekend,wind_speed,day_of_week,wind_compass_direction,sea_level_pressure,air_temperature,day_of_month,dew_temperature,hour,month,meter,building_id,site_id,floor_count,square_feet
0,11.000,0,6.000,0,0.000,4,0.000,1019.500,25.000,1,20.000,0,1,0,0,0,nan,7432
1,15.000,0,6.000,0,0.000,4,0.000,1019.500,25.000,1,20.000,0,1,0,1,0,nan,2720
2,28.000,0,6.000,0,0.000,4,0.000,1019.500,25.000,1,20.000,0,1,0,2,0,nan,5376
3,17.000,0,6.000,0,0.000,4,0.000,1019.500,25.000,1,20.000,0,1,0,3,0,nan,23685
4,44.000,0,6.000,0,0.000,4,0.000,1019.500,25.000,1,20.000,0,1,0,4,0,nan,116607


In [28]:
y = train['log_meter_reading']

In [29]:
y.shape

(20125605,)

In [30]:
y.iloc[100:105]

100   0.000
101   0.000
102   0.000
103   3.191
104   0.318
Name: log_meter_reading, dtype: float32

In [31]:
print("Min value of log_meter_reading is:", y.min())
print("Median value of log_meter_reading is:", y.median())
print("Max value of log_meter_reading is:", y.max())

Min value of log_meter_reading is: 0.0
Median value of log_meter_reading is: 4.3788967
Max value of log_meter_reading is: 16.902212


In [32]:
# All features 
all_features = ['building_id', 'meter', 'site_id',
                'square_feet', 'floor_count', 'air_temperature',
                'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
                'sea_level_pressure', 'wind_speed', 'wind_compass_direction',
                'le_primary_use', 'building_age', 'month', 'day_of_week',
                'day_of_month', 'hour', 'is_weekend']

In [33]:
X2 = train[all_features]

In [34]:
X2.head()

,building_id,meter,site_id,square_feet,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_speed,wind_compass_direction,le_primary_use,building_age,month,day_of_week,day_of_month,hour,is_weekend
0,0,0,0,7432,nan,25.000,6.000,20.000,nan,1019.500,0.000,0.000,0,11.000,1,4,1,0,0
1,1,0,0,2720,nan,25.000,6.000,20.000,nan,1019.500,0.000,0.000,0,15.000,1,4,1,0,0
2,2,0,0,5376,nan,25.000,6.000,20.000,nan,1019.500,0.000,0.000,0,28.000,1,4,1,0,0
3,3,0,0,23685,nan,25.000,6.000,20.000,nan,1019.500,0.000,0.000,0,17.000,1,4,1,0,0
4,4,0,0,116607,nan,25.000,6.000,20.000,nan,1019.500,0.000,0.000,0,44.000,1,4,1,0,0


In [35]:
X2_train = X2[:int(3 * X2.shape[0] / 4)]
X2_valid = X2[int(3 * X2.shape[0] / 4):]

In [36]:
X2_train.shape

(15094203, 19)

In [37]:
X2_valid.shape

(5031402, 19)

In [38]:
# split train and validation set into 75 and 25 percent sequentially
X_train = X[:int(3 * X.shape[0] / 4)]
X_valid = X[int(3 * X.shape[0] / 4):]

y_train = y[:int(3 * y.shape[0] / 4)]
y_valid = y[int(3 * y.shape[0] / 4):]

In [39]:
# make sure train and validation sets shape align
print("Shape of the training set is: ", X_train.shape)
print("Shape of the validation set is: ", X_valid.shape)
print("Shape of the training labels are: ", y_train.shape)
print("Shape of the validation labels are: ", y_valid.shape)

Shape of the training set is:  (15094203, 18)
Shape of the validation set is:  (5031402, 18)
Shape of the training labels are:  (15094203,)
Shape of the validation labels are:  (5031402,)


In [40]:
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true-y_pred)**2))

In [41]:
baseline_guess = np.median(y_train)

print('The baseline guess is a score of %0.2f' % baseline_guess)
print("Baseline Performance on the valid set: RMSE = %0.4f" % rmse(y_valid, baseline_guess))

The baseline guess is a score of 4.38
Baseline Performance on the valid set: RMSE = 2.1122


In [42]:
# impute missing values 

# function to impute missing values with median values of the training set
def my_median_imputer(df_train, df_valid):
    for col in df_train.columns:
        col_median = df_train[col].median()
        df_train.fillna(col_median, inplace=True)
        df_valid.fillna(col_median, inplace=True)
    return df_train, df_valid

In [43]:
X_train, X_valid = my_median_imputer(X_train, X_valid)

In [44]:
# make sure there are no NA values left.
X_train.isnull().sum()

building_age              0
le_primary_use            0
cloud_coverage            0
is_weekend                0
wind_speed                0
day_of_week               0
wind_compass_direction    0
sea_level_pressure        0
air_temperature           0
day_of_month              0
dew_temperature           0
hour                      0
month                     0
meter                     0
building_id               0
site_id                   0
floor_count               0
square_feet               0
dtype: int64

In [45]:
X_valid.isnull().sum()

building_age              0
le_primary_use            0
cloud_coverage            0
is_weekend                0
wind_speed                0
day_of_week               0
wind_compass_direction    0
sea_level_pressure        0
air_temperature           0
day_of_month              0
dew_temperature           0
hour                      0
month                     0
meter                     0
building_id               0
site_id                   0
floor_count               0
square_feet               0
dtype: int64

In [46]:
def fit_evaluate_model(model, X_train, y_train, X_valid, Y_valid):
    model.fit(X_train, y_train)
    y_predicted = model.predict(X_valid)
    return sqrt(mean_squared_error(y_valid, y_predicted))

#### Linear Regression

In [47]:
# Linear Regression 
linear_regression = LinearRegression()
lr_rmse = fit_evaluate_model(linear_regression, X_train, y_train, X_valid, y_valid)
print("RMSE of the linear regression model is:", lr_rmse)

RMSE of the linear regression model is: 2.0159557932652885


#### Light Gradient Boosting 

In [48]:
# create model apply fit_evaluate_model
lgbm_regressor = lgb.LGBMRegressor()
lgbm_rmse = fit_evaluate_model(lgbm_regressor, X_train, y_train, X_valid, y_valid)
print("RMSE of the light gbm regressor is:", lgbm_rmse)

RMSE of the light gbm regressor is: 1.4452167248017687


In [49]:
# Model RMSE after feature revision 
updated_lgbm_rmse = fit_evaluate_model(lgbm_regressor, X2_train, y_train, X2_valid, y_valid)
print("RMSE of the light gbm regressor is:", updated_lgbm_rmse)

RMSE of the light gbm regressor is: 1.4491634270009675


In [50]:
# Since light Gradient Boosting works well with categorical features 
# create categorical features 
categorical_features = ['building_id', 'site_id', 'meter',
                        'le_primary_use', 'cloud_coverage', 
                        'wind_compass_direction',
                        'day_of_week', 'hour','is_weekend']

In [ ]:
# Now we have 9 numerical and 9 categorical 


